# Error Propagation for SED Modelling
This script is intended to be used to perform error propogation with monte carlo simulations. The intent of this script will be to use a sample galaxy sed from cigale and create variations of through pertubing it with random noise. 

In [42]:
# Import all required packages
import matplotlib.pyplot as plt
import astropy.units as u
import numpy as np
import pandas as pd
import os
from astLib import astSED
import astropy.io.fits as fits
from carf import * # custom module for functions relating to the project
import matplotlib.path as mpath
import seaborn as sns

# refresh

# So that we can change the helper functions without reloading the kernel
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
# Load in all of the filters
# Filters
pb_U_path = os.path.join('datasets', 'Filters', 'Generic_Johnson.U.dat')
pb_V_path = os.path.join('datasets', 'Filters', 'Generic_Johnson.V.dat')
pb_J_path = os.path.join('datasets', 'Filters', '2MASS_2MASS.J.dat')


pb_U = astSED.Passband(pb_U_path, normalise=False)
pb_V = astSED.Passband(pb_V_path, normalise=False)
pb_J = astSED.Passband(pb_J_path, normalise=False)

In [44]:
full_cdfs_ids = pd.read_csv('datasets/zfourge/full_CDFS_ids.csv')
full_cosmos_ids = pd.read_csv('datasets/zfourge/full_COSMOS_ids.csv')
full_uds_ids = pd.read_csv('datasets/zfourge/full_UDS_ids.csv')


In [45]:

# Using dataframes is wildly inefficient. 
# A most robust approach would be to use only the 2 columns we are interested in, specified in the function call
# and then read these in as numpy arrays


def get_n_seds(df, n, field, restframe=False, all=False):
    # Select n galaxies
    
    df_list = []
    names = []
    redshifts = []
    if all==False:
        selected_galaxies = df.sample(n)
    else: 
        selected_galaxies = df
        
    # Reset the index
    selected_galaxies = selected_galaxies.reset_index(drop=True)
    
    # name 
    gal_name = selected_galaxies['id'].astype(str)
    
    # field
    gal_field = field#selected_galaxies['field'].astype(str)
    
    
    names = gal_field + '_' + gal_name
    gal_redshift = selected_galaxies['zpk'].astype(float)

    # Now we will read in the fits files for these galaxies

    for i in range(len(selected_galaxies)):
        path = 'datasets\\full_zfourge_decomposed\\'+ str(gal_field).lower() +'_best_models_fits\\'
        name = str(gal_name[i])+'_best_model.fits'

        galaxy_path = os.path.join(path, name)
        with fits.open(galaxy_path) as data:
            df = pd.DataFrame(np.array(data[1].data).byteswap().newbyteorder())
        
        # Convert to angstroms
        df['wavelength'] = df['wavelength']*10

        if restframe:
            df['Snu'] = df['Fnu']*10**-3 # milliJanksys to Janksys <- J = ergs/(s*(cm^2)*(s^-1))
            # F_nu currently has a frequency dependence, convert to nuFnu by multiplying the the frequency associated
            # with the wavelength, as we are in angstroms, we can use the formula c = f*lambda
            
            
            # This should prevent any issues, but check
            freq = (3*10**18)/df['wavelength'] # in Hz
            # multiply the Snu * nu to get nuSnu
            df['nuSnu'] = df['Snu']*freq
            # Restframe the values of wavelength
            df['wavelength'] = df['wavelength'] / (1 + gal_redshift[i]) # we redshift the values of of wavelength
            # now calculate a new frequency, based on the new wavelength
            freq = (3*10**18)/df['wavelength'] # in Hz
            # divide the nuSnu by the new frequency to get the restframed values
            df['Snu'] = df['nuSnu']/freq
            
            # Convert flux values
            df['Flambda'] = df['Snu']*(3*10**-5)/(df['wavelength']**2) # S_nu to F_lambda <- angstroms 
            
        else:
            # Convert flux values
            df['Snu'] = df['Fnu']*10**-3 # milliJanksys to Janksys <- J = ergs/(s*(cm^2)*(s^-1))
            df['Flambda'] = df['Snu']*(3*10**-5)/(df['wavelength']**2) # S_nu to F_lambda <- angstroms 
            
        
            
            
        redshift_Val = gal_redshift[i]
        redshifts.append(redshift_Val)        
        
        

        
        # For simplicity, just create some extra columns
        df['lambda (Angstroms)'] = df['wavelength']
        df['Total Flux (erg/s/cm^2/Angstrom)'] = df['Flambda']
        
        
        
        df_list.append(df)
        
        
        plt.loglog(df['wavelength'], df['Flambda'])
    plt.xlabel('Wavelength (Angstroms)')
    plt.ylabel('Flux (Fnu)')
    #plt.xlim(1e3, 1e5)
    plt.ylim(1e-30, 1e-2)
    plt.title('SED of galaxies')
    plt.legend()
    plt.show()
    
    print(len(df_list))
    
    return df_list, names, redshifts


In [46]:
np.random.seed(42)

def get_n_seds(df, n, field, restframe=False, all=False):

    # Select n galaxies (only the necessary columns)
    if all:
        selected_indices = np.arange(len(df))
    else:
        selected_indices = np.random.choice(len(df), n, replace=False)

    names = []
    redshifts = []
    df_list = []
    
    print(selected_indices)
    
    for i in selected_indices:
        name = f"{field}_{df['id'][i]}"

        redshift = df['zpk'][i]
        names.append(name)
        redshifts.append(redshift)

        # Load FITS file (using only the necessary columns)
        path = f'datasets\\full_zfourge_decomposed\\{field.lower()}_best_models_fits\\'
        file = f"{df['id'][i]}_best_model.fits"

        with fits.open(os.path.join(path, file)) as data:
            wavelength = data[1].data['wavelength'] * 10  # Convert to Angstroms
            fnu = data[1].data['Fnu']

        # Convert and rest-frame data (in-place operations for efficiency)
        snu = fnu * 1e-3  # milliJansky to Jansky
        flambda = snu * 3e-5 / wavelength**2

        if restframe:
            wavelength /= (1 + redshift)
            freq = 3e18 / wavelength
            snu = snu * freq / freq  # Avoid division by zero
            flambda = snu * 3e-5 / wavelength**2

        # Create DataFrame (only with necessary columns)
        df_galaxy = pd.DataFrame({
            'lambda (Angstroms)': wavelength,
            'Total Flux (erg/s/cm^2/Angstrom)': flambda
        })

        df_list.append(df_galaxy)
        
        # plotting, keep if needed
        #plt.loglog(df_galaxy['lambda (Angstroms)'], df_galaxy['Total Flux (erg/s/cm^2/Angstrom)'])

    # Plotting (only once after the loop)
    # plt.xlabel('Wavelength (Angstroms)')
    # plt.ylabel('Flux (Fnu)')
    # plt.ylim(1e-30, 1e-2)
    # plt.title('SED of galaxies')
    # plt.legend()
    # plt.show()

    return df_list, names, redshifts


In [47]:
# Get one SED for testing from the CDFS field
test_sed_list, names, redshifts = get_n_seds(full_cdfs_ids, 1, 'CDFS', restframe=True, all=False)

TypeError: get_n_seds() got an unexpected keyword argument 'random_seed'

In [ ]:
# Now we can safely sele